# Titanic

## Exercícios

1 Leia os dados train.csv usando pandas

2 Remova a coluna com nomes de pessoas

3 A coluna Cabin contém uma letra e um número. Crie uma nova coluna chamada deck usando apenas a letra de cada registro presente na coluna Cabin.

4 Substitua os valores ausentes pela média de valores da coluna

5 Os valores da coluna "embarked" sao os seguintes: C = Cherbourg, Q = Queenstown, S = Southampton

6 Crie um CSV com essas uma coluna nome_cidade_de_embarque: C = Cherbourg, Q = Queenstown, S = Southampton e mais uma coluna chamada população com valores hipotéticos, faça um merge entre o CSV train.csv e esse novo CSV com os nomes das cidades. Implemente um filtro de passageiros por população da cidade de embarque

In [49]:
import pandas as pd
import numpy as np
import sqlite3


url = "https://raw.githubusercontent.com/advinstai/python-datascience/master/Exercicios/titanic/train.csv"
csv_data = pd.read_csv(url, encoding="utf-8", sep=",")
print("\n\nShape do arquivo lido do csv:", csv_data.shape)
print("\n\nSample:\n", csv_data.sample(5))

# connection to the database, create if database not exists
con = sqlite3.connect('database.db', timeout=10)
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS titanic;")

# create a sql statement from dataframe data automatically
def getType(column, dfcolumn):
    aux = " INTEGER" if (dfcolumn.dtype == np.float64 or dfcolumn.dtype == np.int64) else " TEXT"
    return column + aux
columns = [getType(column, csv_data[column]) for column in csv_data.columns]  
sql_create = "CREATE TABLE titanic (" + ', '.join(columns) + ");"
print("\n\nComando para criar tabela:\n", sql_create)
#cur.execute(sql_create)

# alternatively one can create a new table directly from csv file using
csv_data.to_sql('titanic', con, if_exists='append', index=False)
db_data = pd.read_sql_query("SELECT * FROM titanic;", con)
print("\n\nShape do arquivo lido do database:", db_data.shape)

con.commit()
con.close()
!fuser database.db # check if all processes were terminated



Shape do arquivo lido do csv: (891, 12)


Sample:
      PassengerId  Survived  Pclass  \
470          471         0       3   
721          722         0       3   
538          539         0       3   
18            19         0       3   
689          690         1       1   

                                                  Name     Sex   Age  SibSp  \
470                                  Keefe, Mr. Arthur    male   NaN      0   
721                          Jensen, Mr. Svend Lauritz    male  17.0      1   
538                           Risien, Mr. Samuel Beard    male   NaN      0   
18   Vander Planke, Mrs. Julius (Emelia Maria Vande...  female  31.0      1   
689                  Madill, Miss. Georgette Alexandra  female  15.0      0   

     Parch  Ticket      Fare Cabin Embarked  
470      0  323592    7.2500   NaN        S  
721      0  350048    7.0542   NaN        S  
538      0  364498   14.5000   NaN        S  
18       0  345763   18.0000   NaN        S  
689      1   

In [50]:
#2 Remova a coluna com nomes de pessoas

print("Shape antes: ", csv_data.shape)
csv_data = csv_data.drop('Name', axis=1)
print(csv_data)

Shape antes:  (891, 12)
     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0              1         0       3    male  22.0      1      0   
1              2         1       1  female  38.0      1      0   
2              3         1       3  female  26.0      0      0   
3              4         1       1  female  35.0      1      0   
4              5         0       3    male  35.0      0      0   
..           ...       ...     ...     ...   ...    ...    ...   
886          887         0       2    male  27.0      0      0   
887          888         1       1  female  19.0      0      0   
888          889         0       3  female   NaN      1      2   
889          890         1       1    male  26.0      0      0   
890          891         0       3    male  32.0      0      0   

               Ticket     Fare Cabin Embarked  
0           A/5 21171   7.2500   NaN        S  
1            PC 17599  71.2833   C85        C  
2    STON/O2. 3101282   7.9250   NaN   

In [51]:
#3 A coluna Cabin contém uma letra e um número. 
#Crie uma nova coluna chamada deck usando apenas a letra de cada registro presente na coluna Cabin.

# add a new column using loc
csv_data.loc[csv_data['Cabin'].notnull(), 'Deck'] = csv_data['Cabin'].str[0]

# add a new column using filters
csv_data['DeckNum'] = csv_data[csv_data['Cabin'].notnull()]['Cabin'].apply(lambda x: x[1:])
print(csv_data)
#csv_data = csv_data.loc[csv_data['Cabin'].notnull()[0], 'Deck']

     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0              1         0       3    male  22.0      1      0   
1              2         1       1  female  38.0      1      0   
2              3         1       3  female  26.0      0      0   
3              4         1       1  female  35.0      1      0   
4              5         0       3    male  35.0      0      0   
..           ...       ...     ...     ...   ...    ...    ...   
886          887         0       2    male  27.0      0      0   
887          888         1       1  female  19.0      0      0   
888          889         0       3  female   NaN      1      2   
889          890         1       1    male  26.0      0      0   
890          891         0       3    male  32.0      0      0   

               Ticket     Fare Cabin Embarked Deck DeckNum  
0           A/5 21171   7.2500   NaN        S  NaN     NaN  
1            PC 17599  71.2833   C85        C    C      85  
2    STON/O2. 3101282   

In [52]:
#4 Substitua os valores ausentes pela média de valores da coluna

print(csv_data.isnull().sum())
csv_data_filled = csv_data.fillna(csv_data.mean())
print(csv_data_filled.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Deck           687
DeckNum        687
dtype: int64
PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Deck           687
DeckNum        687
dtype: int64


In [65]:
# 5 Os valores da coluna "embarked" sao os seguintes: C = Cherbourg, Q = Queenstown, S = Southampton
# 6 Crie um CSV com essas uma coluna nome_cidade_de_embarque: 
#   C = Cherbourg, Q = Queenstown, S = Southampton e mais uma coluna chamada população com valores hipotéticos, 
#    faça um merge entre o CSV train.csv e esse novo CSV com os nomes das cidades. 
#    Implemente um filtro de passageiros por população da cidade de embarque

cities = pd.DataFrame({'city_id': ['C','Q','S'], 
                       'city': ['Cherbourg','Queenstown','Southhampton'], 
                       'pop': [234234,134123,52452]})

print(cities)

merged = pd.merge(csv_data_filled, cities, left_on='Embarked', right_on='city_id', how='left')
print(merged.drop('city_id', axis=1).shape)
print(merged.drop('city_id', axis=1))

  city_id          city     pop
0       C     Cherbourg  234234
1       Q    Queenstown  134123
2       S  Southhampton   52452
(891, 15)
     PassengerId  Survived  Pclass     Sex        Age  SibSp  Parch  \
0              1         0       3    male  22.000000      1      0   
1              2         1       1  female  38.000000      1      0   
2              3         1       3  female  26.000000      0      0   
3              4         1       1  female  35.000000      1      0   
4              5         0       3    male  35.000000      0      0   
..           ...       ...     ...     ...        ...    ...    ...   
886          887         0       2    male  27.000000      0      0   
887          888         1       1  female  19.000000      0      0   
888          889         0       3  female  29.699118      1      2   
889          890         1       1    male  26.000000      0      0   
890          891         0       3    male  32.000000      0      0   

         